In [1]:
from importlib import reload
def reload_all():
    import tsm; reload(tsm)
    import _keys; reload(_keys)
    import blizzard; reload(blizzard)
    import config; reload(config)
    import kvstore; reload(kvstore)
    import snapshot; reload(snapshot)
    import elemental_arbitrage; reload(elemental_arbitrage)
    import bliz_tsm_join; reload(bliz_tsm_join)
    import crafting; reload(crafting)
    import kvstore; reload(kvstore)
    import crafting; reload(crafting)

In [2]:
reload_all()

In [3]:
from snapshot import SnapshotProcessor
from tsm import auction_house_snapshot
from config import tsm_region_id
from config import tsm_realm_id
from config import tsm_ah_id
from config import tsm_cache_dir

def tsm_ah_snapper():
    return auction_house_snapshot(tsm_region_id, tsm_realm_id, tsm_ah_id)

tsm_ah_snap = SnapshotProcessor(tsm_ah_snapper, cache_dir=tsm_cache_dir)
tsm_ah = tsm_ah_snap.get(max_age_seconds=3000)

In [4]:
from snapshot import SnapshotProcessor
from blizzard import auction_data
from blizzard import auction_summary
from config import blizzard_realm_id
from config import blizzard_ah_id
from config import blizzard_cache_dir

def bliz_ah_snapper():
    return auction_data(blizzard_realm_id, blizzard_ah_id)

bliz_ah_snap = SnapshotProcessor(bliz_ah_snapper, cache_dir=blizzard_cache_dir)
bliz_ah = bliz_ah_snap.get(max_age_seconds=3000)

In [5]:
from kvstore import InefficientKVStore
from blizzard import ItemLookup
from config import blizzard_item_cache
from config import blizzard_item_reverse_cache

items = ItemLookup(
    InefficientKVStore(blizzard_item_cache),
    InefficientKVStore(blizzard_item_reverse_cache),
)

In [6]:
from bliz_tsm_join import item_info_getter
item_info = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=3000)
item_info_current = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=0)

In [19]:
reload_all()

from crafting import Recipes

r = Recipes(items)
i = r.ingredients

r.recipe_from_strings(
    "2 flask of the frost wyrm",
    "5 icethorn + 5 lichbloom + frost lotus + enchanted vial",

)
r.recipe_from_strings(
    "firecracker salmon",
    "glacial salmon + northern spices",
)
r.recipe_from_strings(
    "elixir of greater agility",
    "sungrass + goldthorn + crystal vial",
)

(UUID('f4ffbf3e-9b51-41cf-81e3-6a2da4cdab1a'),
 [elixir of greater agility],
 [sungrass] + [crystal vial] + [goldthorn])

In [20]:
from crafting import crafting_price
crafting_price(item_info, r.lookup(i("firecracker salmon"))[0])

{'profit': 4.18,
 'recipe': ([glacial salmon] + [northern spices], [firecracker salmon]),
 'input_price_method': <function crafting.gold.<locals>._gold(item_inf)>,
 'output_price_method': <function crafting.gold.<locals>._gold(item_inf)>,
 'input_breakdown': {'[glacial salmon]': {'count': 1,
   'price': 1.49,
   'total': 1.49},
  '[northern spices]': {'count': 1, 'price': 4.42, 'total': 4.42},
  'TOTAL': 5.91},
 'output_breakdown': {'[firecracker salmon]': {'count': 1,
   'price': 10.09,
   'total': 10.09},
  'TOTAL': 10.09}}

In [9]:
item_info("elixir of greater agility")

{'id': 9187,
 'name': 'elixir of greater agility',
 'num_auctions': 12,
 'quantity': 12,
 'weight_sell': {'gold': 5.26, 'copper': 52607.0},
 'avg_sell': {'gold': 5.26, 'copper': 52607.0},
 'max': {'gold': 5.26, 'copper': 52607},
 'p80': {'gold': 5.26, 'copper': 52607},
 'p50': {'gold': 5.26, 'copper': 52607},
 'p20': {'gold': 5.26, 'copper': 52607},
 'wp80': {'gold': 5.26, 'copper': 52607},
 'wp50': {'gold': 5.26, 'copper': 52607},
 'wp20': {'gold': 5.26, 'copper': 52607},
 'min': {'gold': 5.26, 'copper': 52607},
 'realm_market_value': {'gold': 2.2, 'copper': 21981},
 'realm_historical': {'gold': 2.16, 'copper': 21647},
 'region_historical': {'gold': 0.99, 'copper': 9896},
 'region_avg_sale_price': {'gold': 0.68, 'copper': 6788},
 'sale_pct': 25.7,
 'sold_per_day': 44,
 'headroom': 159,
 'market_skew_pct': 1.54,
 'auction_skew_pct': 139.33}

In [10]:
item_info("firecracker salmon")

{'id': 34767,
 'name': 'firecracker salmon',
 'num_auctions': 12,
 'quantity': 144,
 'weight_sell': {'gold': 150.4, 'copper': 1504042.6666666667},
 'avg_sell': {'gold': 111.26, 'copper': 1112581.8333333333},
 'max': {'gold': 194.11, 'copper': 1941078},
 'p80': {'gold': 194.11, 'copper': 1941060},
 'p50': {'gold': 140.8, 'copper': 1407984},
 'p20': {'gold': 44.0, 'copper': 440000},
 'wp80': {'gold': 194.11, 'copper': 1941060},
 'wp50': {'gold': 176.0, 'copper': 1759980},
 'wp20': {'gold': 48.53, 'copper': 485280},
 'min': {'gold': 44.0, 'copper': 440000},
 'realm_market_value': {'gold': 10.1, 'copper': 101036},
 'realm_historical': {'gold': 10.95, 'copper': 109497},
 'region_historical': {'gold': 5.22, 'copper': 52158},
 'region_avg_sale_price': {'gold': 5.44, 'copper': 54400},
 'sale_pct': 46.6,
 'sold_per_day': 569,
 'headroom': 1077,
 'market_skew_pct': -7.73,
 'auction_skew_pct': 335.49}

In [11]:
28*20*.6453

361.368

In [12]:
item_info("book of glyph mastery")

{'id': 45912,
 'name': 'book of glyph mastery',
 'num_auctions': 3,
 'quantity': 3,
 'weight_sell': {'gold': 18.12, 'copper': 181216.66666666666},
 'avg_sell': {'gold': 18.12, 'copper': 181216.66666666666},
 'max': {'gold': 24.37, 'copper': 243651},
 'p80': {'gold': 24.37, 'copper': 243651},
 'p50': {'gold': 15.0, 'copper': 150000},
 'p20': {'gold': 15.0, 'copper': 149999},
 'wp80': {'gold': 24.37, 'copper': 243651},
 'wp50': {'gold': 15.0, 'copper': 150000},
 'wp20': {'gold': 15.0, 'copper': 149999},
 'min': {'gold': 15.0, 'copper': 149999},
 'realm_market_value': {'gold': 17.43, 'copper': 174255},
 'realm_historical': {'gold': 22.76, 'copper': 227635},
 'region_historical': {'gold': 15.29, 'copper': 152855},
 'region_avg_sale_price': {'gold': 11.23, 'copper': 112264},
 'sale_pct': 66.5,
 'sold_per_day': 186,
 'headroom': 276,
 'market_skew_pct': -23.45,
 'auction_skew_pct': -13.92}

In [13]:
from elemental_arbitrage import print_elemental_arbitrage_report

print_elemental_arbitrage_report(tsm_ah, items)

DEFINITELY   BS eternal-life (unit profit: 17280,headroom: 3476)
DEFINITELY   BS eternal-earth (unit profit: 46670,headroom: 3514)
DEFINITELY   BS eternal-air (unit profit: 34657,headroom: 2452)
DEFINITELY   BS eternal-fire (unit profit: 64569,headroom: 2133)
DEFINITELY   BS eternal-water (unit profit: 27891,headroom: 5757)
DEFINITELY   BS eternal-shadow (unit profit: 59346,headroom: 3756)
SKIP        primal-life
SKIP        primal-earth
SKIP        primal-air
SKIP        primal-fire
SKIP        primal-water
SKIP        primal-shadow
DEFINITELY   BS essence-eternal (unit profit: 277739,headroom: 413)
DEFINITELY   BS essence-cosmic (unit profit: 895782,headroom: 953)
DEFINITELY   BS essence-nether (unit profit: 165503,headroom: 413)
DEFINITELY   BS essence-planar (unit profit: 66685,headroom: 803)
DEFINITELY   BS essence-astral (unit profit: 98553,headroom: 598)
SKIP        essence-magic
DEFINITELY   BS essence-mystic (unit profit: 69529,headroom: 792)


In [14]:
item_info("crystallized water")

{'id': 37705,
 'name': 'crystallized water',
 'num_auctions': 26,
 'quantity': 182,
 'weight_sell': {'gold': 192.66, 'copper': 1926563.7747252746},
 'avg_sell': {'gold': 242.72, 'copper': 2427187.346153846},
 'max': {'gold': 3110.14, 'copper': 31101411},
 'p80': {'gold': 2.3, 'copper': 22970},
 'p50': {'gold': 2.19, 'copper': 21950},
 'p20': {'gold': 0.66, 'copper': 6588},
 'wp80': {'gold': 7.92, 'copper': 79192},
 'wp50': {'gold': 2.2, 'copper': 22000},
 'wp20': {'gold': 2.19, 'copper': 21920},
 'min': {'gold': 0.22, 'copper': 2194},
 'realm_market_value': {'gold': 0.44, 'copper': 4404},
 'realm_historical': {'gold': 0.79, 'copper': 7897},
 'region_historical': {'gold': 0.56, 'copper': 5583},
 'region_avg_sale_price': {'gold': 0.56, 'copper': 5584},
 'sale_pct': 75.9,
 'sold_per_day': 4508,
 'headroom': 5757,
 'market_skew_pct': -44.23,
 'auction_skew_pct': -50.18}

In [15]:
found_sholazar = {"adder's": 3*20+15, "goldclover": 2*20+9, "frost lotus": 3, "deadnettle": 20+3, "tiger lily": 2*20+2, "crystallized life": 10*5+3}
rates_per_minute = {k: v/50 for (k, v) in found_sholazar.items()}
rates_per_minute

{"adder's": 1.5,
 'goldclover': 0.98,
 'frost lotus': 0.06,
 'deadnettle': 0.46,
 'tiger lily': 0.84,
 'crystallized life': 1.06}